In [8]:
import pycountry

def get_country_name(alpha2_code):
    try:
        country = pycountry.countries.get(alpha_2=alpha2_code.upper())
        return country.name if country else None
    except LookupError:
        return None
def get_country_code(country_name):
    try:
        # Search by name (case-insensitive)
        country = next(
            c for c in pycountry.countries 
            if c.name.lower() == country_name.lower()
        )
        return country.alpha_2
    except StopIteration:
        return None


In [4]:
from rdflib import Dataset

d = Dataset(store="Oxigraph")
d.open("./data/book.db/")

1

In [ ]:
to_change = dict()

for r in d.query(r"""
PREFIX schema: <http://schema.org/>
SELECT ?name
WHERE {
  ?country a schema:Country .
  ?country schema:name ?name .
  FILTER(STRLEN(?name) > 2)
}
"""):
    cc = get_country_code(r.get("name"))
    if(cc != None):
        to_change[cc] = r.get("name")
    
    # print(r.get("name"), ":" , get_country_code(r.get("name")))

In [ ]:
from string import Template


find_tmp = Template(r"""
PREFIX schema: <http://schema.org/>
DELETE {
  ?country schema:name "$cc".
}
INSERT {
  ?country schema:name "$fullname" .
  ?country schema:alternateName "$cc" .
}
WHERE {
  ?country a schema:Country .
  ?country schema:name "$cc".
}
""")

for key in to_change.keys():
    q = find_tmp.substitute(fullname = to_change[key], cc=key)
    d.update(q)

In [21]:
for r in d.query(r"""
PREFIX schema: <http://schema.org/>
SELECT ?name
WHERE {
  ?country a schema:Country .
  ?country schema:name ?name .
  FILTER(STRLEN(?name) = 2)
}
"""):
    print(r)

(rdflib.term.Literal('SR', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('CL', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('CI', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('SZ', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('UK', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('GE', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('BW', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('CU', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('KZ', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),)
(rdflib.term.Literal('HN', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#st

In [22]:
d.close()